# Train judge embedding using the task of predicting ruling label

In [1]:
# Phu, Andrea and Watcher
# 2018 Spring
import numpy as np
import pandas as pd
import time
import os
import pickle
import string
import psutil
import sklearn

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from ggplot import *

from sklearn.manifold import TSNE
import math

import seaborn as sns


# Phu, Andrea and Watcher
# 2018 Spring
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch import FloatTensor, LongTensor
import pickle
import torch.utils.data as data_utils
from random import shuffle
from sklearn.utils import shuffle as skshuffle
import random

torch.manual_seed(44)
random.seed(44)
np.random.seed(44)


/home/chewang/.conda/envs/judge_emb/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/chewang/.conda/envs/judge_emb/lib/python3.6/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/chewang/.conda/envs/judge_emb/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
processed_data_path = '/data/Dropbox/judge_embedding_data_sp18'
all_data_save_path = os.path.join(processed_data_path,"finalized_all_data","all_data_dict.pkl")
all_data_support_save_path = os.path.join(processed_data_path,"finalized_all_data","all_data_dict_support.pkl")
all_data_df_save_path = os.path.join(processed_data_path,"finalized_all_data","all_data_df.h5")
finished_embedding_folder_path = os.path.join(processed_data_path,'finished_judge_embedding')
plot_save_path = '/data/Dropbox/judge_embedding_data_sp18/plots'

## read in data for each case

In [3]:
all_data_dict = pickle.load(open(all_data_save_path,"rb"))
topic_glove_emb = all_data_dict['topic_glove_emb'][0]
judgeId2Index = all_data_dict['judge_id_to_index'][0]
judgeIndex2Id = all_data_dict['judge_index_to_id'][0]
all_data_df = all_data_dict['data_df']
all_data_df[:4]

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss,opinion_vector,judge_embed_index,Circuit,date,affirmdummy,judge_decision
0,XB0PMNQNB5G0,1891,contentMajOp,"[0, 202, 1268, 13, 0, 4600, 3, 0, 247954, 128,...","PARDEE, DON A.",1830.0,Civil Procedure,4.0,"[-0.07445931306292275, 0.05843820821632856, -0...",0,5,1891-12-07,0.0,0
1,XFL757,1891,contentMajOp,"[15, 116, 4, 9238, 7, 22547, 3, 4092, 1219, 0,...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.07101038203845275, 0.029874279749715626, -...",1,6,1891-10-06,0.0,0
2,XB0PMRQNB5G0,1891,contentMajOp,"[215, 6, 68193, 3527, 255, 9670, 5, 12375, 3, ...","PARDEE, DON A.",1830.0,Contracts,7.0,"[-0.059691082627907024, 0.10039284279069777, -...",0,5,1891-12-07,0.0,0
3,XFL7KI,1891,contentMajOp,"[5, 0, 935, 4951, 5, 8151, 0, 4490, 3, 0, 305,...","LOCKE, JAMES WILLIAM",1410.0,Civil Procedure,4.0,"[-0.13210430623456784, 0.12352108086419746, -0...",2,5,1891-11-27,0.0,0


In [4]:
# we now read in the judge bio data
judge_bio_path = os.path.join(processed_data_path,"JudgesBioReshaped_TOUSE.dta")
judge_bio_df = pd.read_stata(judge_bio_path)


In [5]:
pd.options.display.max_columns = 999
judge_bio_df

,songername,judgeidentificationnumber,Term,id,AppointmentDate,TerminationDate,courtname,courttype,abarating,AbaUpdate,SenateConfirmationdate,RecessAppointDate,birthday,birthmonth,birthyear,deathmonth,deathday,deathyear,JudgeAssignedCourt,judgelastname,judgefirstname,judgemiddlename,suffix,Circuitjudge,SeatState,retirementfromactiveservice,dateaschiefjudgebegin,presidentname,seconddateaschiefjudgebegin,senatevoicevote,senatevoteayesnays,Committeeactiondate,Degree1,Degree2,Degree3,Degree4,Degree5,degreeyear1,degreeyear2,degreeyear3,degreeyear4,degreeyear5,Committeeaction,ageon,vicelastnamepredecessor,vicefirstnamepredecessor,hearings,reportnumber,authorizationlegislation,placeofbirthcity,placeofbirthstate,placeofdeathcity,placeofdeathstate,deathdate,party_Updated,dateaschiefjudgeend,seconddateaschiefjudgeend,renominatingpresidentname,partyaffiliationofrenompres,referraldatejudicialcom,degree,degree_law,degree_other,district,nominationdatesenateexecutivej,genderNew,raceorethnicity,partyaffiliationofpresident,recessappointmentdate,committeeaction,committeeactiondate,senatevotedateconfirmationdate,commissiondate,dateoftermination,terminationspecificreason,nameofschool1,degree1,nameofschool2,degree2,nameofschool3,degree3,nameofschool4,degree4,nameofschool5,degree5,employmenttextfield,bankruptcyandmagistrateservice,Nominationdate,District_Circuit,party,pres,left,state,city,appres,aba,congresi,unityi,hdem,hrep,sdem,srep,hother,sother,x_dem,x_republican,x_instate_ba,x_elev,x_unity,x_aba,x_crossa,x_pfedjdge,x_pindreg1,x_plawprof,x_pscab,x_pcab,x_pusa,x_pssenate,x_paag,x_psp,x_pslc,x_pssc,x_pshouse,x_psg,x_psgo,x_psenate,x_psatty,x_pprivate,x_pmayor,x_plocct,x_phouse,x_pgov,x_pda,x_pcc,x_pccoun,x_pausa,x_pasatty,x_pag,x_pada,x_pgovt,x_llm_sjd,x_protestant,x_evangelical,x_mainline,x_noreligion,x_catholic,x_jewish,x_black,x_nonwhite,x_female,x_jd_public,x_ba_public,x_b10s,x_b20s,x_b30s,x_b40s,x_b50s,x_pbank,x_pmag,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,SchoolState3,Public3,Private3,SchoolState4,Public4,Private4,SchoolState5,Public5,Private5,TonotInclude,idtocorrrect,ayear,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon40orless,x_ageon70ormore,pres_correct2,political_party,x_pago,startdate
0,"ABRAMS, RONNIE",3419,1,NaN,2012-03-23,NaT,"u. s. district court, southern district of new...",usdc,Qualified,,2012-03-22,NaT,NaN,NaN,1968.0,NaN,NaN,NaN,,ABRAMS,RONNIE,,,2,NY,,NaN,Barack Obama,NaN,No,96/2,2011-11-03,2.0,5.0,NaN,NaN,NaN,1990,1993,,,NaN,1.0,44.0,Kaplan,Lewis A.,10/04/2011,,,New York,NY,,,NaT,NaN,NaN,NaN,,,07/28/2011,,,,0208,07/28/2011,F,White,Democratic,,Reported With Recommendation,11/03/2011,03/22/2012,03/23/2012,,,Cornell University,B.A.,Yale Law School,J.D.,,,,,,,"Law clerk, Hon. Thomas P. Griesa, U.S. Distric...",,18836.0,,NaN,94.0,NaN,NaN,NaN,1.0,3.0,112TH,2.0,193.0,242.0,52,47,0.0,4.0,1.0,0.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NY,0.0,1.0,CT,0.0,1.0,,NaN,NaN,,NaN,NaN,,NaN,NaN,0.0,NaN,2012.0,1.0,0.0,0.0,0.0,0.0,NaN,,0.0,2012-03-23
1,"ABRUZZO, MATTHEW J.",1,1,2.0,1936-02-15,1971-05-28,"u. s. district court, eastern district of new ...",usdc,,,1936-02-12,NaT,30.0,4.0,1889.0,5.0,28.0,1971.0,Brooklyn,ABRUZZO,MATTHEW,T.,,2,NY,02/15/1966,NaN,Franklin D. Roosevelt,NaN,Yes,,1936-02-11,6.0,NaN,NaN,NaN,NaN,1910,,,,NaN,1.0,47.0,new,,,,49 Stat. 659,Brooklyn,NY,Potomac,MD,1971-05-28,1.0,NaN,NaN,,,02/03/1936,,,,0207,02/03/1936,M,White,Democratic,,Reported With Recommendation,02/11/1936,02/12/1936,02/15/1936,05/28/1971,Death,Brooklyn Law School,LL.B.,,,,,,,,,"Private practice, Brooklyn, New York, 1910-1936",,-8733.0,0207,1.0,14.0,3.0,32.0,3220.0,1.0,6.0,74TH,0.0,322.0,103.0,69,25,0.0,2.0,1.0,0.0,0.0,NaN,1.0,0.0,0.0,NaN,1.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,

In [6]:
judge_bio_df['raceorethnicity'].unique()

array(['White', 'Hispanic', 'African American', ' ', 'Hispanic/Asian Am.',
       'African Am./Hispanic', 'American Indian', 'Asian American',
       'Pac. Isl./Asian Am.', 'Pac. Isl./White'], dtype=object)

In [7]:
judge_bio_df['partyaffiliationofpresident'].unique()

array(['Democratic', 'Republican', 'Reassignment',
       'Jeffersonian Republican', 'Federalist', ' ', 'Assignment', 'Whig'],
      dtype=object)

In [8]:
judge_bio_df['genderNew'].unique()

array(['F', 'M'], dtype=object)

## get citation embeddings

In [9]:
citation_folder_path = os.path.join(processed_data_path,"citation_graph_data_node2vec")
citation_emb_path = os.path.join(citation_folder_path,"citation.emb")
citation_case2index_path = os.path.join(citation_folder_path,"case_id_to_index.pickle")
citation_name2index_path = os.path.join(citation_folder_path,"citation_name_to_index.pickle")
citation_emb_fpt = open(citation_emb_path,"r")
citation_emb_lines = citation_emb_fpt.readlines()
citation_caseid2Index = pickle.load(open(citation_case2index_path,"rb"))
citation_name2Index = pickle.load(open(citation_name2index_path,"rb"))

In [10]:
def citation_txt_to_numpy(lines):
    R, C = lines[0].strip().split()
    R = int(R)
    C = int(C)
    token_list = []
    for r in range(1,1+R):
        line = lines[r]
        tokens = line.strip().split()
        token_list.append(tokens[1:])
    return np.array(token_list).astype(float)

In [11]:
citation_emb = citation_txt_to_numpy(citation_emb_lines)

In [12]:
def add_citation_emb_to_df(df):
    # this function will add citation data to all_data_df
    df['citation'] = None
    n_instance = df.shape[0]
    citation_emb_dim = citation_emb.shape[1]
    case_no_cit_count = 0
    for i in range(n_instance):
        case = df.iloc[i]
        caseid = case['caseid']
        if caseid in citation_caseid2Index:
            citation_index = citation_caseid2Index[caseid]
            citation_vec = citation_emb[citation_index]
        else:
            case_no_cit_count+=1
            citation_vec = np.zeros(citation_emb_dim)
        
        df.iat[i,df.columns.get_loc('citation')] = citation_vec
    print("number of citation not found:", case_no_cit_count)

In [13]:
citation_emb.shape

(350840, 128)

In [14]:
add_citation_emb_to_df(all_data_df)

number of citation not found: 64438


In [15]:
all_data_df.shape[0]

295507

In [16]:
len(citation_caseid2Index.keys())

281593

## There is a number of cases where their corresponding citations are not found. We simply use all 0 citation embeddings for them

In [17]:
pd.options.display.max_rows = 20
all_data_df

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss,opinion_vector,judge_embed_index,Circuit,date,affirmdummy,judge_decision,citation
0,XB0PMNQNB5G0,1891,contentMajOp,"[0, 202, 1268, 13, 0, 4600, 3, 0, 247954, 128,...","PARDEE, DON A.",1830.0,Civil Procedure,4.0,"[-0.07445931306292275, 0.05843820821632856, -0...",0,5,1891-12-07,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,XFL757,1891,contentMajOp,"[15, 116, 4, 9238, 7, 22547, 3, 4092, 1219, 0,...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.07101038203845275, 0.029874279749715626, -...",1,6,1891-10-06,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,XB0PMRQNB5G0,1891,contentMajOp,"[215, 6, 68193, 3527, 255, 9670, 5, 12375, 3, ...","PARDEE, DON A.",1830.0,Contracts,7.0,"[-0.059691082627907024, 0.10039284279069777, -...",0,5,1891-12-07,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,XFL7KI,1891,contentMajOp,"[5, 0, 935, 4951, 5, 8151, 0, 4490, 3, 0, 305,...","LOCKE, JAMES WILLIAM",1410.0,Civil Procedure,4.0,"[-0.13210430623456784, 0.12352108086419746, -0...",2,5,1891-11-27,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,XB0OI7QNB5G0,1891,contentMajOp,"[5214, 10, 894, 2091, 4297, 21, 0, 14485, 1268...","NELSON, SUSAN RICHARD",3339.0,NaN,NaN,"[-0.09034438144397865, 0.06449958495506274, -0...",3,8,NaN,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,XB0PHFQNB5G0,1891,contentMajOp,"[0, 377, 202, 3, 14963, 386, 92, 3, 745, 98, 0...","PARDEE, DON A.",1830.0,Torts,7.0,"[-0.06601334912248623, 0.019215826325411318, -...",0,5,1891-11-27,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,XB0PJRQNB5G0,1891,contentMajOp,"[202, 1268, 21, 0, 14485, 38, 14, 60389, 187, ...","BRUCE, JOHN",292.0,Corporate Law,7.0,"[-0.11053901197255082, 0.10235485285668748, -0...",4,5,1891-12-07,1.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,XFL6PE,1891,contentMajOp,"[15, 116, 4, 18428, 0, 305, 4, 0, 1399, 202, 3...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.05526258051118215, 0.049538111821086264, -...",1,6,1891-10-06,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,XFL6LS,1891,contentMajOp,"[3, 133, 10550, 10, 1609, 400000, 0, 58, 84, 4...","SAGE, GEORGE READ",2089.0,Patent Law,7.0,"[-0.09545400352665524, 0.10511163229642662, -0...",5,6,1891-10-06,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,XB0PNFQNB5G0,1891,contentMajOp,"[0, 2536, 3, 323, 1853, 128, 3, 50, 196, 7091,...","PARDEE, DON A.",1830.0,Mortgages & Liens,7.0,"[-0.08174452291734276, 0.08172237472717521, -0...",0,5,1891-12-07,1.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## We will do a normalization of doc vectors, using center by topic-year
## That means for each doc vector, we reduce from it the topic-year vector it is related to.
## E.g. if a doc vector is from 1981 and topic of civil procedure, then we reduce it by the average vector of all vectors that are from 1981 and of topic civil procedure.

In [18]:
def get_year_topic_dict(df):
    # given all data df, give the year-topic average vectors
    # in the form of a year-topic dictionary
    # key is year-topic string, and value is a 300 dim vector
    # which is the average of all the cases with that year and topic
    
    yearTopicDict = {}
    yearTopicCount = {}
    
    n_instance = df.shape[0]
    for i in range(n_instance):
        case = df.iloc[i]
        year = case['year']
        topic = case['topic']
        yearTopic = str(year)+"-"+str(topic)
        opinion_vector = case['opinion_vector']
        
        if yearTopic not in yearTopicDict:
            yearTopicDict[yearTopic] = np.zeros(300) + opinion_vector
            yearTopicCount[yearTopic] = 1
        else:
            yearTopicDict[yearTopic] += opinion_vector
            yearTopicCount[yearTopic] += 1
            
    # now we accumulated all the cases
    # we do an average for each dictionary entry
    for k in yearTopicDict.keys():
        yearTopicDict[k] /= yearTopicCount[k]
        
    return yearTopicDict

def add_centered_vec_to_df(df,year_topic_dict,verbose=0):
    starttime = time.time()
    df['centered_opinion_vec'] = None
    n_instance = df.shape[0]
    for i in range(n_instance):
        if verbose and (i==2 or i%int(n_instance/20)==0 ) :
            print(i,time.time()-starttime)
        case = df.iloc[i]
        year = case['year']
        topic = case['topic']
        yearTopic = str(year)+"-"+str(topic)
        meanTopicYear = year_topic_dict[yearTopic]
        opinion_vector = case['opinion_vector']    
        centered_vec = opinion_vector - meanTopicYear
        
        df.iat[i,df.columns.get_loc('centered_opinion_vec')] = centered_vec
        
year_topic_dict = get_year_topic_dict(all_data_df)
add_centered_vec_to_df(all_data_df,year_topic_dict,verbose=1)

0 0.0061588287353515625
2 0.007008790969848633
14775 4.912052631378174
29550 9.412665367126465
44325 13.751875877380371
59100 18.17250370979309
73875 23.232064962387085
88650 28.572127103805542
103425 32.97863483428955
118200 38.555691719055176
132975 43.00514841079712
147750 47.95689630508423
162525 53.051677942276
177300 59.05548167228699
192075 63.73627161979675
206850 68.9733533859253
221625 73.22633290290833
236400 77.61494421958923
251175 82.65959334373474
265950 88.092524766922
280725 92.42345809936523
295500 97.97340703010559


## Next we train a model to do the ruling prediction task

In [19]:
# first split the data

def train_val_test_split(data_df,number_judges,train_ratio=0.8,val_ratio=0.1,verbose=0,toshuffle=True):
    starttime= time.time()
    sorted_all_data = data_df.sort_values(by='judge_embed_index')
    train_indexes = []
    val_indexes = []
    test_indexes = []
    currentiloc = 0
    for judge_index in range(number_judges):
        if verbose and judge_index%500 == 0:
            print(judge_index,time.time()-starttime)
        
        cases_of_this_judge = sorted_all_data.loc[sorted_all_data['judge_embed_index'] == judge_index]
        number_cases = cases_of_this_judge.shape[0]
        n_of_train = int(number_cases*train_ratio)
        n_of_val = int(number_cases*val_ratio)
        
        nextiloc = currentiloc+number_cases
        
        indexes = [i for i in range(currentiloc, nextiloc)]
        shuffle(indexes)
        
        train_indexes += indexes[:n_of_train]
        val_indexes += indexes[n_of_train:n_of_train+n_of_val]
        test_indexes += indexes[n_of_train+n_of_val:]
        
        currentiloc = nextiloc
    return skshuffle(data_df.loc[train_indexes]),skshuffle(data_df.loc[val_indexes]),skshuffle(data_df.loc[test_indexes])

data_train, data_val, data_test = train_val_test_split(all_data_df,2099,verbose=1)

0 1.5883769989013672
500 2.03888201713562
1000 2.4861629009246826
1500 2.936947822570801
2000 3.361992597579956


# now we get data in tensor form

In [20]:
all_data_df

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss,opinion_vector,judge_embed_index,Circuit,date,affirmdummy,judge_decision,citation,centered_opinion_vec
0,XB0PMNQNB5G0,1891,contentMajOp,"[0, 202, 1268, 13, 0, 4600, 3, 0, 247954, 128,...","PARDEE, DON A.",1830.0,Civil Procedure,4.0,"[-0.07445931306292275, 0.05843820821632856, -0...",0,5,1891-12-07,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0058334392332427265, -0.013141758486908628,..."
1,XFL757,1891,contentMajOp,"[15, 116, 4, 9238, 7, 22547, 3, 4092, 1219, 0,...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.07101038203845275, 0.029874279749715626, -...",1,6,1891-10-06,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.009282370257712724, -0.04170568695352156, -..."
2,XB0PMRQNB5G0,1891,contentMajOp,"[215, 6, 68193, 3527, 255, 9670, 5, 12375, 3, ...","PARDEE, DON A.",1830.0,Contracts,7.0,"[-0.059691082627907024, 0.10039284279069777, -...",0,5,1891-12-07,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,XFL7KI,1891,contentMajOp,"[5, 0, 935, 4951, 5, 8151, 0, 4490, 3, 0, 305,...","LOCKE, JAMES WILLIAM",1410.0,Civil Procedure,4.0,"[-0.13210430623456784, 0.12352108086419746, -0...",2,5,1891-11-27,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.051811553938402366, 0.05194111416096027, -..."
4,XB0OI7QNB5G0,1891,contentMajOp,"[5214, 10, 894, 2091, 4297, 21, 0, 14485, 1268...","NELSON, SUSAN RICHARD",3339.0,NaN,NaN,"[-0.09034438144397865, 0.06449958495506274, -0...",3,8,NaN,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,XB0PHFQNB5G0,1891,contentMajOp,"[0, 377, 202, 3, 14963, 386, 92, 3, 745, 98, 0...","PARDEE, DON A.",1830.0,Torts,7.0,"[-0.06601334912248623, 0.019215826325411318, -...",0,5,1891-11-27,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,XB0PJRQNB5G0,1891,contentMajOp,"[202, 1268, 21, 0, 14485, 38, 14, 60389, 187, ...","BRUCE, JOHN",292.0,Corporate Law,7.0,"[-0.11053901197255082, 0.10235485285668748, -0...",4,5,1891-12-07,1.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,XFL6PE,1891,contentMajOp,"[15, 116, 4, 18428, 0, 305, 4, 0, 1399, 202, 3...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.05526258051118215, 0.049538111821086264, -...",1,6,1891-10-06,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.025030171784983325, -0.02204185488215092, -..."
8,XFL6LS,1891,contentMajOp,"[3, 133, 10550, 10, 1609, 400000, 0, 58, 84, 4...","SAGE, GEORGE READ",2089.0,Patent Law,7.0,"[-0.09545400352665524, 0.10511163229642662, -0...",5,6,1891-10-06,0.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.010078752686018252, -0.009731982682180201, ..."
9,XB0PNFQNB5G0,1891,contentMajOp,"[0, 2536, 3, 323, 1853, 128, 3, 50, 196, 7091,...","PARDEE, DON A.",1830.0,Mortgages & Liens,7.0,"[-0.08174452291734276, 0.08172237472717521, -0...",0,5,1891-12-07,1.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.015055386863120815, -0.020542366529214207, ..."


In [21]:
def df_to_Tensor(df,topic_glove_emb,verbose=0):
    # use this to convert a dataframe to torch tensor
    # this is the predict ruling label version
    
    # WE DO NOT ADD JUDGE EMBEDDING HERE, THAT WILL BE HANDLED BY THE MODEL
    
    feature_dim = 300+300+128+1 # 300 for opinion vec, 300 for topic, 128 for citation, 
    # and 1 value to indicate which judge
    
    X = np.zeros((df.shape[0],feature_dim))
    y = np.zeros(df.shape[0]) # y is the ruling label
    
    for i in range(df.shape[0]):
        if verbose and i%10000==0:
            print(i)
        
        data_entry = df.iloc[i]
        
        # add opinion vector to X
        X[i,:300] = data_entry['centered_opinion_vec']
        
        # add topic vector to X
        topic = data_entry['topic']
        topic = str.lower(str(topic)) 
        
        if topic not in topic_glove_emb: # deal with any unknown topic
            topic = "<UNK>"
            
        X[i,300:600] = topic_glove_emb[topic]
        
        # add citation vector to X
        X[i,600:600+128] = data_entry['citation']
        
        X[i,-1] = data_entry['judge_embed_index'] # in the model, the model needs to pick a judge vector
        # according to this value
        
        # set y
        decision = data_entry['judge_decision']
        y[i] = decision
        
    return FloatTensor(X),LongTensor(y)

In [22]:
X_train, y_train = df_to_Tensor(data_train,topic_glove_emb,1)
X_val, y_val = df_to_Tensor(data_val,topic_glove_emb,1)
X_test, y_test = df_to_Tensor(data_test,topic_glove_emb,1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
0
10000
20000
0
10000
20000
30000


## Now we have X_train, y_train and in Tensor form, we can now create the model and start our training

In [24]:
BATCH_SIZE = 128
train_dataset = data_utils.TensorDataset(data_tensor=X_train,target_tensor=y_train)
train_loader = data_utils.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)

## The ruling label prediction requires the model to put judge embedding as part of the input
## The model will first concatenate the judge embedding with the input X it get from data loader and then put them through the layers.

In [49]:
class Judge_emb_model(nn.Module):
    def __init__(self, input_dim, hidden_layer_dim, embedding_dim, num_judges):
        super(Judge_emb_model,self).__init__()
        # since output is one of 2 classes, output dim is 2 
        
        self.judge_embedding = Variable(torch.FloatTensor(num_judges,embedding_dim),requires_grad=True)  # H x J
        
        # input is m x D
        self.linear1 = nn.Linear(input_dim-1+embedding_dim,hidden_layer_dim) # D x H  # -1 because that is for judge
        self.dropout1 = nn.Dropout(p=0.5)
        self.linear2 = nn.Linear(hidden_layer_dim,hidden_layer_dim) # H x 2
        self.dropout2 = nn.Dropout(p=0.5)
        self.linear3 = nn.Linear(hidden_layer_dim,2) # H x 2
        # the output is m x J
        
        self.init_weights()

    def forward(self, X):
        # first concatenate the X and the judge emb
        judge_indexes = X[:,-1].data.numpy().astype(int).tolist()
        
        X_combined = torch.cat([X[:,:-1],self.judge_embedding[judge_indexes]],1)
        
        out = F.relu(self.linear1(X_combined))
        out = self.dropout1(out)
        out = F.relu(self.linear2(out))
        out = self.dropout2(out)
        
        out = self.linear3(out)
        
        # now we have m x J matrix, for m data points, we can do log softmax
        log_prob = F.log_softmax(out,dim=1)
        return log_prob # for each opinion data, this is probability of which judge writes this opinion
    
    def init_weights(self):
        linear_layers = [self.linear1,self.linear2,self.linear3]
        for layer in linear_layers:
            layer.weight.data.normal_(0.0,0.1)
        self.judge_embedding.data.normal_(0.0,0.1)


In [50]:
INPUT_DIM = X_train.shape[1]
HIDDEN_DIM = 300
EMBED_DIM = 300
number_judges = 2099
model = Judge_emb_model(input_dim=INPUT_DIM,hidden_layer_dim=HIDDEN_DIM,
                        embedding_dim=EMBED_DIM,num_judges=number_judges)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(model.parameters())+[model.judge_embedding],lr=0.005)

In [ ]:
N_EPOCH = 8
TRAIN_SIZE = train_dataset.data_tensor.shape[0]
print("Training data size",TRAIN_SIZE)
train_losses = []
val_losses = []

X_val_var = Variable(X_val)
y_val_var = Variable(y_val)
model.eval()
y_pred_val = model.forward(X_val_var)
val_loss = criterion(y_pred_val,y_val_var)
print("initial val loss",val_loss.data[0])
startTime = time.time()
model.train()

for i_epoch in range(N_EPOCH):
    epoch_train_loss = 0
    num_batches_per_epoch = int(TRAIN_SIZE/BATCH_SIZE)
    for i_batch,(X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        
        X_var, y_var = Variable(X_batch),Variable(y_batch)
        
        y_pred = model.forward(X_var)
        loss = criterion(y_pred,y_var)
        loss.backward()
        
        optimizer.step()
#         if i_batch % 2000 == 0:
#             print(i_epoch,i_batch,loss.data[0])
        epoch_train_loss += loss.data[0]
        
    # after each epoch
    
    X_val_var = Variable(X_val)
    y_val_var = Variable(y_val)
    model.eval()
    y_pred_val = model.forward(X_val_var)
    val_loss = criterion(y_pred_val,y_val_var)
    ave_train_loss = epoch_train_loss/num_batches_per_epoch
    print("epoch",i_epoch,"ave_train_loss",
          ave_train_loss,"validation loss:",val_loss.data[0],time.time()-startTime)
    val_losses.append(val_loss.data[0])
    train_losses.append(ave_train_loss)
    model.train()

Training data size 235456
initial val loss 0.9120796918869019
epoch 0 ave_train_loss 0.6892493115682327 validation loss: 0.6715168356895447 74.31359672546387
epoch 1 ave_train_loss 0.6765952902750323 validation loss: 0.6715830564498901 154.3624346256256


In [52]:
train_losses

[0.6892493115682327,
 0.6765952902750323,
 0.676532369356171,
 0.6763865735104319,
 0.6757554731140842,
 0.6762669820575496,
 0.6758265734172633,
 0.6749772365939818]

In [54]:
val_losses

[0.6715168356895447,
 0.6715830564498901,
 0.6740204095840454,
 0.6757417917251587,
 0.6723085641860962,
 0.673294723033905,
 0.670312762260437,
 0.6680342555046082]

In [55]:
## now we save the trained embedding to disk
judge_emb = model.judge_embedding.data

In [58]:
judge_emb = judge_emb.numpy()

In [59]:
judge_emb.shape

(2099, 300)

In [60]:
save_pred_ruling_trained_path = os.path.join(finished_embedding_folder_path,"pred_ruling_trained_emb.pkl")
pickle.dump(judge_emb,open(save_pred_ruling_trained_path,"wb"))